# Giới thiệu 
- Notebook này là phần được ghi chú lại trong quá học khóa học [rag-from-scratch](https://github.com/langchain-ai/rag-from-scratch?tab=readme-ov-file) về việc xây dựng mô hình RAG từ đầu
- Trong phần này chủ yếu giới thiệu về cách xây dựng một pipeline cơ bản nhất cho một hệ thống RAG, pipeline bao gồm 3 phần chính như sau: 
<p align="center">
    <img src="../doc/image/basic-image.png" alt="basic-pipeline" width="400"/>
</p>




- Trong đó : 
1. Indexing: Phần dữ liệu nền tảng sẽ được thu thập và tách, sau đó được indexing vào trong một vector database
2. Retrieval: phần này xử lý dữ liệu người dùng (user query) sẽ được index vào trong vector database để tìm kiếm được k chunks liên quan nhất tới query 
3. Generation : phần này chuyển đổi yêu cầu của người dùng và lượng dữ liệu của đã được trích xuất tù phần retrieval để tạo ra response với trả lời câu hỏi của người dùng dựa trên kiến thức được trích xuất.   

# Triển khai Basic pipeline
- Triển khai một pipeline của RAG dựa trên : 
    1. Mô hình LLM và Embedding : Gemini
    2. Dữ liệu indexing : [wiki - Faker](https://en.wikipedia.org/wiki/Faker_(gamer)) 
    3. Trích xuất để trả lời câu hỏi "How many World Championship titles has Faker won in his League of Legends career?" 

## Setup 

In [3]:
# import thư viện cần thiết 
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader, PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
import tiktoken

USER_AGENT environment variable not set, consider setting it to identify your requests.
d:\WorkSpace_Trung\AI\simple-rag\rag\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import os 
from dotenv import load_dotenv

load_dotenv()

GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")
LANGCHAIN_TRACING_V2 = os.getenv("LANGCHAIN_TRACING_V2")
LANGCHAIN_ENDPOINT = os.getenv("LANGCHAIN_ENDPOINT")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")

## Indexing 

<p align="center">
    <img src="../doc/image/document-loading.png" alt="basic-pipeline" width="400"/>
</p>


- Document sẽ được load và lưu trữ vào trong một vector database, việc này giúp tìm kiếm và truy xuất thông tin một cách hiệu quả hơn 
- Langchain hỗ trợ cho nhiều nguồn dữ liệu được chuyển đổi thông qua module [Document Loaders](https://python.langchain.com/docs/integrations/document_loaders/)
- Một số nguồn thường dùng là dữ liệu trực tiếp từ trang web hoặc các file định dạng PDF 

In [ ]:
# loader thông qua nguồn là một file pdf 
loader = PyPDFLoader(
    file_path = "data\TTHCM1.pdf"
)

docs = loader.load()


# loader thông qua nguồn là trang web
loader = WebBaseLoader(
    web_paths=("https://en.wikipedia.org/wiki/Faker_(gamer)",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("mw-body-content")
        )
    ),
)
docs = loader.load()

- Sau khi dữ liệu được load từ các nguồn trên, để trích xuất hoạt động hiệu quả và nhanh chóng hơn, dữ liệu sẽ được chia thành từng đoạn nhỏ theo chunk_size và chunk_overlap. Dữ liệu sẽ được chia nhỏ cho tới khi vừa dủ hoặc nhỏ hơn với chunk_size.

In [ ]:
# Split chia đoạn document được load
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
splits[0]

- Để máy tính, các mô hình có thể hiểu được ngữ cảnh, và ngôn ngữ, nó cần được biểu diễn dưới dạng số. Các kí tự được mã hóa này được gọi là Token. Token giống như một chuỗi các kí tự trong đoạn text và sau đó được mã hóa trở thành một biểu diễn đưới dạng số 

In [ ]:
# Documents
question = "What kinds of pets do I like?"
document = "My favorite pet is a cat."

# đếm số lượng token của một câu 
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string(question, "cl100k_base")

- Mô hình embedding giúp chuyển đối các kí tự dạng chữ, biểu diễn chúng dưới dạng số, chuỗi số, giúp mô hình có thể hiểu được. Các biểu diễn này thông qua mô hình vẫn có thể nắm bắt được các mối quan hệ giữa các thành phần có trong câu. Sau khi chuyển đổi này, các mô hình còn hỗ trợ việc tính toán độ tương đồng giữa các câu, token thông qua tính toán các vector
<p align="center">
    <img src="../doc/image/embedding-model.png" alt="basic-pipeline" width="400"/>
</p>

In [ ]:
embd = GoogleGenerativeAIEmbeddings(model="models/embedding-001",  google_api_key = os.environ['GOOGLE_API_KEY'] )
query_result = embd.embed_query(question)
document_result = embd.embed_query(document)
len(query_result)

- Kết thúc quá trình xử lý dữ liệu, các vector sẽ được lưu trữ vào trong một hệ cơ sở dữ liệu dành riêng. Hệ cơ sở này thực hiện việc chuyển đổi các chuỗi đã được split từ trước dó thông qua mô hình embedding để lưu trữ chúng dưới dạng các vector. 

<p align="center">
    <img src="../doc/image/preprocessing-pipeline.png" alt="basic-pipeline" width="400"/>
</p>

In [ ]:
# Embedding đoạn dữ liệu vào trong vector database thông qua mô hình Gemini
vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001",  google_api_key = GOOGLE_API_KEY ))

retriever = vectorstore.as_retriever()

## Retrieval 
- Indexing bản chất là quá trình xử lý dữ liệu và chuyển đổi thông qua mô hình embedding. 
- Retrieval là quá trình trích xuất dữ liệu, thông tin liên quan với thông tin đầu vào từ trong vector database. 
- Có rất nhiều thuật toán tìm kiếm thông tin gần nhất trong một vector database (cosine similarity)

<p align="center">
    <img src="../doc/image/retrieval-algo.png" alt="basic-pipeline" width="400"/>
</p>

In [ ]:
# set đầu ra của kết quả retrieval luôn là 1 
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

# tìm kiếm doc liên quan tới câu hỏi 
docs = retriever.get_relevant_documents("How many World Championship titles has Faker won in his League of Legends career?")
docs

## Generation 
- Tiếp tục với pipeline ở ban đầu, sau khi một số lượng document được trích xuất ra liên quan tới câu hỏi, tổng hợp chúng sẽ được đi qua một mẫu prompt và một LLM để trả lời câu hỏi của người dùng.

In [ ]:
# Prompt được lấy từ hub có sẵn
prompt = hub.pull("rlm/rag-prompt")

# Load LLm model
llm = ChatGoogleGenerativeAI(model = 'gemini-1.5-pro-latest',
                                       temperature=0.2,
                                       api_key= GOOGLE_API_KEY)

# Post-processing hậu xử lý văn bản
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Question
rag_chain.invoke("How many World Championship titles has Faker won in his League of Legends career?")